In [27]:
import urllib, time, json, csv
import numpy as np
import pandas as pd

In [28]:
write_path = 'auctions.json'
#api_key = 'CharlesZ-11ce-4517-8df3-becc92494006'
api_key = 'MichaelA-757e-4b18-8eeb-2a0a915831f3'

In [29]:
# for finding api, findCompletedItems calls
def finding_data(page):
    # parameter and values for call
    paramKeys = ['OPERATION-NAME', 'SERVICE-VERSION','SECURITY-APPNAME','RESPONSE-DATA-FORMAT','categoryId', 'paginationInput.pageNumber']
    paramVals = ['findCompletedItems','1.7.0',api_key,'JSON&REST-PAYLOAD','9355', str(page)]

    # call url
    url_finding = 'http://svcs.ebay.com/services/search/FindingService/v1?'
    
    # constructing the url
    url = url_finding
    for i in range(len(paramKeys)):
        url += paramKeys[i] + '=' + paramVals[i] + '&'
    
    # loading the return response
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    
    # parse data for selected auctions
    return data['findCompletedItemsResponse'][0]['searchResult'][0]['item']



In [30]:
# for shopping api, getSingleItem calls
def item_data(item_id):
    # parameter and values for call
    paramKeys = ['callname', 'responseencoding', 'appid', 'siteid', 'version', 'ItemID', 'IncludeSelector']
    paramVals = ['GetSingleItem', 'JSON', api_key, '0', '515', item_id, 'Details']
    
    # call url
    url_item = 'http://open.api.ebay.com/shopping?'
    
    # constructing the url
    url = url_item
    for i in range(len(paramKeys)):
        url += paramKeys[i] + '=' + paramVals[i] + '&'
        
    # loading the return response, keep trying until success
    while True:
        try:
            response = urllib.urlopen(url)
            data = json.loads(response.read())
            break
        except ValueError:
            #time.sleep(1)
            pass
        
    
    #parse data for selected auction
    return data['Item']


In [33]:
# for shopping api, getUserProfile calls
def user_data(user_id):
    # parameter and values for call
    paramKeys = ['callname', 'responseseencoding', 'appid', 'sideid', 'version', 'UserID', 'IncludeSelector']
    paramVals = ['GetUserProfile', 'JSON', api_key, '0', '525', user_id, 'FeedbackHistory']
    
    # call url
    url_user = 'http://open.api.ebay.com/shopping?'
    
    # constructing the url
    url = url_user
    for i in range(len(paramKeys)):
        url += paramKeys[i] + '=' + paramVals[i] + '&'
    
    # loading the return response 
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    
    #parse data for selected auction
    return data

In [216]:
# finding api
# loop backwards to avoid conflicts
total_data = []
with open('test.json',"w") as outfile:
    for i in range(0,100):
        data = finding_data(100-i)
        time.sleep(1) 
        total_data += data
        print "page", 100-i, "complete"
    
    json.dump(total_data,outfile)

page 100 complete
page 99 complete
page 98 complete
page 97 complete
page 96 complete
page 95 complete
page 94 complete
page 93 complete
page 92 complete
page 91 complete
page 90 complete
page 89 complete
page 88 complete
page 87 complete
page 86 complete
page 85 complete
page 84 complete
page 83 complete
page 82 complete
page 81 complete
page 80 complete
page 79 complete
page 78 complete
page 77 complete
page 76 complete
page 75 complete
page 74 complete
page 73 complete
page 72 complete
page 71 complete
page 70 complete
page 69 complete
page 68 complete
page 67 complete
page 66 complete
page 65 complete
page 64 complete
page 63 complete
page 62 complete
page 61 complete
page 60 complete
page 59 complete
page 58 complete
page 57 complete
page 56 complete
page 55 complete
page 54 complete
page 53 complete
page 52 complete
page 51 complete
page 50 complete
page 49 complete
page 48 complete
page 47 complete
page 46 complete
page 45 complete
page 44 complete
page 43 complete
page 42 compl

In [226]:

# get all auctions that we are working with
auctions = set()
with open('test.json') as fd:
    reader = json.load(fd)
    for auction in reader:
        auctions.add(auction['itemId'][0])
        




In [227]:
len(auctions)

9914

In [153]:
test = item_data('381476042249')
type(test)

http://open.api.ebay.com/shopping?callname=GetSingleItem&responseencoding=JSON&appid=CharlesZ-11ce-4517-8df3-becc92494006&siteid=0&version=515&ItemID=381476042249&IncludeSelector=Details&


dict

In [228]:
# shopping api


outfile =  open('auctions_detailed.json', 'w')


In [234]:
list(auctions)[66]

u'262172152634'

In [236]:
counter = 0
total_auc_data = []
for auc in list(auctions):
    try:
        data = item_data(auc)
    except KeyError:
        # auction id doesn't exist? remove auction from set
        auctions.remove(auc)
        #continue
        continue
    
    total_auc_data.append(data)
    counter += 1
    print counter

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [237]:
json.dump(total_auc_data,outfile)
outfile.close()

In [179]:
user_ids = set()
for x in total_auc_data:
    user_ids.add(x['Seller']['UserID'])
    


In [185]:
aucs_ids = set()

for x in total_auc_data:
    aucs_ids.add(x['ItemID'])

In [183]:
user_ids

{u'2jspawn',
 u'811shell',
 u'918_recell',
 u'92_ranks',
 u'accessorie.ssupplies',
 u'adesai',
 u'allmetals71',
 u'alvealedu',
 u'amad_ceda',
 u'amran_0',
 u'barnaclebrain',
 u'bchangsf1',
 u'beddy2',
 u'bgim_enterprises_inc',
 u'blondest1',
 u'brandon82cars',
 u'bronsomurph_0',
 u'bruno2bk',
 u'buzprohiba',
 u'cabinetgallery',
 u'cashinelectronicsandgaming0',
 u'chest_marlo',
 u'christy4lyfe',
 u'cleek-peter',
 u'coachm1k3',
 u'cool_rep',
 u'crawf_shaw',
 u'cybermike24',
 u'danbear777',
 u'dd7714',
 u'destiny_solutions',
 u'dgaudie03',
 u'directbuywireless',
 u'dkthekidd40',
 u'dmm_den',
 u'dshportko',
 u'edmundlatoya',
 u'emassey678',
 u'eu52',
 u'everthingcool',
 u'fairheartcolls',
 u'gabe6390',
 u'galacticom',
 u'galvanism',
 u'ghantechmobile10',
 u'ghvlp',
 u'gizmotrader_contact_us_before_leaving_feedback',
 u'goodbuystuff2',
 u'guatemala58',
 u'happy2290',
 u'heath832',
 u'horsesfarmers',
 u'icb-cellphonesbeyond',
 u'igadgetselectronicsinc',
 u'iiamcorinne',
 u'indigi',
 u'ingra-

In [186]:
len(aucs_ids)

166

In [189]:
aucs_ids1 = set()
for x in auctions:
    aucs_ids1.add(x)
len(aucs_ids1)

166

In [217]:
test1 = set()
for x in total_data:
    test1.add(x['itemId'][0])
len(test1)


9914

9914

In [43]:
# user_list = []
# with open('auctions_detailed.json') as testfile:
#     reader = json.load(testfile)
#     for auct in reader:
#         user_list += auct['Seller']['UserID']
#         print auct['Seller']['UserID']
count=0
userlist = []
user_set = set()
with open('auctions_detailed.json') as testfile:
    reader = json.load(testfile)
    for auct in reader:
        username = auct['Seller']['UserID']
        print username
        count+=1
        userlist.append(username)
#     for auct in reader:
#         user_set.add(auct['Seller']['UserID'])
#         print auct['Seller']['UserID']
print len(userlist)

abtboss
jgmcompsupport
flappy_deal
chocolatecakemu
bstew2525
erlamar2011
domcia_w
dres-andr
berlioth1
huoikt88
wallachee
wallachee
pende_kath
dwrinc
www_cent
vamudder
wise-deals
joagerme0
annearundeljewelrybuyers
sparkles2119
upgradesolution
gazelle-store
rodriguezmujica
barnyardwireless
us2015.fos
toniesoprano
gooverstock
fulllove365
moshe2908
alex1855588
upgradesolution
v7stars
juligate
dylankays
davie-deals
sasha_il
jwiedenman
alloutmobil
partsonlyphones
alex1855588
theepp
mstoyzz
alltechwholesale
wine438
goldpony72
techmiaonline
1theonly
nodzade
twinmom505
sellercortex
alltechwholesale
pawns-not-redeemed
denni.arace
celldays
trimman10
thermokool
wessterling22
shutterblade
ritchmobiles
ajseattle
khangoodguy_228
robin-9002i0
2014happyhong
tonkyota
ecellsolutions
shutterblade
mcchronic_276
mwaura16
antoinettenoz
discountedcelltech
sellingtech13
nbmc299
dwrinc
toyboy321
paubo52
ducklady_101
dinocubs1
thephonguy
ltinius1988
ccclubmis
erika4b
jerryp76
jerryp76
relertech3
fargo10101
gmmad

In [24]:
len(user_set)

5922

In [45]:
len(set(userlist))

5922

In [46]:
count = 0
user_auc_data = []
for user in list(users):
    try:
        data = user_data(user)
    except KeyError:
        # auction id doesn't exist? remove auction from set
        users.remove(user)
        #continue
        continue
    
    user_auc_data.append(data)
    counter += 1
    print counter

ValueError: No JSON object could be decoded